In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import html
import string
from datetime import datetime

## Check if Connection Works

In [2]:
url = 'https://www.eia.gov/dnav/ng/hist/n3010us2A.htm'  # Replace with the URL of the website you want to check

try:
    # Make an HTTP request to the website
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        print(f"Connection successful to {url}")

        # Use Beautiful Soup to parse the HTML content
        soup = BeautifulSoup(response.text, 'lxml')

        # Now you can work with the BeautifulSoup object (soup) to extract information
        # For example, print the title of the webpage
        print("Webpage Title:", soup.title.text)
    else:
        print(f"Failed to connect to {url}. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")

Connection successful to https://www.eia.gov/dnav/ng/hist/n3010us2A.htm
Webpage Title: U.S. Natural Gas Residential Consumption  (Million Cubic Feet)


## Find Table of Information

In [3]:
table = soup.find_all('td', class_=['B3', 'B4'])

In [4]:
type(table)

bs4.element.ResultSet

In [5]:
table

[<td class="B4">  1930's</td>,
 <td class="B3">295,700</td>,
 <td class="B3">294,406</td>,
 <td class="B3">298,520</td>,
 <td class="B3">283,197</td>,
 <td class="B3">288,236</td>,
 <td class="B3">313,498</td>,
 <td class="B3">343,346</td>,
 <td class="B3">371,844</td>,
 <td class="B3">367,772</td>,
 <td class="B3">391,153</td>,
 <td class="B4">  1940's</td>,
 <td class="B3">443,646</td>,
 <td class="B3">442,067</td>,
 <td class="B3">498,537</td>,
 <td class="B3">529,444</td>,
 <td class="B3">562,183</td>,
 <td class="B3">607,400</td>,
 <td class="B3">660,820</td>,
 <td class="B3">802,150</td>,
 <td class="B3">896,348</td>,
 <td class="B3">992,544</td>,
 <td class="B4">  1950's</td>,
 <td class="B3">1,198,369</td>,
 <td class="B3">1,474,725</td>,
 <td class="B3">1,621,966</td>,
 <td class="B3">1,685,503</td>,
 <td class="B3">1,894,248</td>,
 <td class="B3">2,123,952</td>,
 <td class="B3">2,327,564</td>,
 <td class="B3">2,500,269</td>,
 <td class="B3">2,714,251</td>,
 <td class="B3">2,9

## Convert List of Values into DataFrame

In [6]:
#Convert BS object to strings
table_list = [str(tag) for tag in table]

# Convert td elements to string
html_content = ''.join(table_list)

# Parse the HTML content
tree = html.fromstring(html_content)

# Extract all td elements
data_list = tree.xpath("//td/text()")


In [7]:
#remove unseen characters
def remove_unseen_char(val):
    return ''.join(char for char in val if char in string.printable)
cleaned_data_list = [remove_unseen_char(elem) for elem in data_list]

In [8]:
#create lists of years and values for df 
years = []
values = []
for element in cleaned_data_list:
    if "'" in element:
        years.append(int(element[:4]))
        #values.append()
    else:
        years.append(years[-1]+1)
        values.append(element)
years = list(set(years))[:-1]     

In [9]:
df = pd.DataFrame({'Years': years, 'Consumption': values})

## Convert df for Analysis

In [10]:
#convert columns into datetime and int objects
df['Years'] = df['Years'].map(lambda x: datetime(x, 12, 31))
df['Consumption'] = df['Consumption'].str.replace(',', '').astype(int)

In [13]:
df.to_csv('Residential_Consumption_data.csv')